|<h2>Course:</h2>|<h1><a href="https://udemy.com/course/dullms_x/?couponCode=202508" target="_blank">A deep understanding of AI language model mechanisms</a></h1>|
|-|:-:|
|<h2>Part 5:</h2>|<h1>Observation (non-causal) mech interp<h1>|
|<h2>Section:</h2>|<h1>Investigating token embeddings<h1>|
|<h2>Lecture:</h2>|<h1><b>CodeChallenge: BERT v GPT kNN kompetition<b></h1>|

<br>

<h5><b>Teacher:</b> Mike X Cohen, <a href="https://sincxpress.com" target="_blank">sincxpress.com</a></h5>
<h5><b>Course URL:</b> <a href="https://udemy.com/course/dullms_x/?couponCode=202508" target="_blank">udemy.com/course/dullms_x/?couponCode=202508</a></h5>
<i>Using the code without the course may lead to confusion or errors.</i>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import matplotlib_inline.backend_inline
matplotlib_inline.backend_inline.set_matplotlib_formats('svg')

In [ ]:
# load BERT tokenizer and model
from transformers import BertTokenizer, BertModel

model = BertModel.from_pretrained('bert-base-uncased')
tokenizerB = BertTokenizer.from_pretrained('bert-base-uncased')
embeddingsB = model.embeddings.word_embeddings.weight.detach().numpy()

In [ ]:
# pretrained GPT-2 model and tokenizer
from transformers import GPT2Model,GPT2Tokenizer
model = GPT2Model.from_pretrained('gpt2')
tokenizerG = GPT2Tokenizer.from_pretrained('gpt2')
embeddingsG = model.wte.weight.detach().numpy()

# Exercise 1: kNN-based synonym-searching of "ring"

In [ ]:
# define a normalized "seed" vector
seedword = 'ring'

# check in BERT
seedidxB = tokenizerB.encode(seedword,add_special_tokens=False)

# check in GPT
seedidxG = tokenizerG.encode(seedword)

print(f'In BERT: "{seedword}" has index {seedidxB}')
print(f'In GPT2: "{seedword}" has index {seedidxG}')

In [ ]:
# Euclidean distances
eucDist_bert = np.sqrt(np.sum( (embeddingsB-embeddingsB[seedidxB,:])**2 ,axis=1))
eucDist_gpt2 = np.sqrt(np.sum( (embeddingsG-embeddingsG[seedidxG,:])**2 ,axis=1))

# visualize the distributions
plt.figure(figsize=(10,4))
yB,xB = np.histogram(eucDist_bert,bins=90,density=True)
yG,xG = np.histogram(eucDist_gpt2,bins=90,density=True)

plt.plot(xG[:-1],yG,linewidth=2,label='GPT2')
plt.plot(xB[:-1],yB,linewidth=2,label='BERT')
plt.legend()
plt.gca().set(xlim=[min(xB.min(),xG.min()),max(xB.max(),xG.max())],
              xlabel='Distances',ylabel='Density',title=f'Distances from "{seedword}"')
plt.show()

In [ ]:
# sort and get top k
k = 15
topKidx_Bert = np.argsort(eucDist_bert)[:k] # [1:k+1] to exclude trivial self-distance
topKidx_gpt2 = np.argsort(eucDist_gpt2)[:k]

# and print
print('|        BERT        |        GPT2          |')
print('|--------------------|----------------------|')
for b,g in zip(topKidx_Bert,topKidx_gpt2):
  print(f'  {tokenizerB.decode([b]):>10} ({eucDist_bert[b]:.2f})  |  ({eucDist_gpt2[g]:4.2f}) {tokenizerG.decode([g])}')

# Exercise 2: Using normalized vectors

In [ ]:
# normalize
embeddingsBnorm = embeddingsB / np.linalg.norm(embeddingsB, axis=1, keepdims=True)
embeddingsGnorm = embeddingsG / np.linalg.norm(embeddingsG, axis=1, keepdims=True)

# Euclidean distances
eucDist_bert = np.sqrt(np.sum( (embeddingsBnorm-embeddingsBnorm[seedidxB,:])**2 ,axis=1))
eucDist_gpt2 = np.sqrt(np.sum( (embeddingsGnorm-embeddingsGnorm[seedidxG,:])**2 ,axis=1))


# visualize the distributions
plt.figure(figsize=(10,4))
yB,xB = np.histogram(eucDist_bert[np.nonzero(eucDist_bert)],bins=90,density=True)
yG,xG = np.histogram(eucDist_gpt2[np.nonzero(eucDist_gpt2)],bins=90,density=True)

plt.plot(xG[:-1],yG,linewidth=2,label='GPT2')
plt.plot(xB[:-1],yB,linewidth=2,label='BERT')
plt.legend()
plt.gca().set(xlim=[min(xB.min(),xG.min()),max(xB.max(),xG.max())],
              xlabel='Distances',ylabel='Density',title=f'Distances from "{seedword}" with vector normalization')
plt.show()

In [ ]:
# sort and get top k
k = 15
topKidx_Bert = np.argsort(eucDist_bert)[:k] # [1:k+1] to exclude trivial self-distance
topKidx_gpt2 = np.argsort(eucDist_gpt2)[:k]

# and print
print('|        BERT        |        GPT2          |')
print('|--------------------|----------------------|')
for b,g in zip(topKidx_Bert,topKidx_gpt2):
  print(f'  {tokenizerB.decode([b]):>10} ({eucDist_bert[b]:.2f})  |  ({eucDist_gpt2[g]:.2f}) {tokenizerG.decode([g])}')

# Exercise 3: The importance of emptiness

In [ ]:
# no new code here :P

# Exercise 4: Multitoken words in GPT2

In [ ]:
# define a normalized "seed" vector
seedword = 'beauty'

# check in BERT
seedidxB = tokenizerB.encode(seedword,add_special_tokens=False)

# check in GPT
seedidxG = tokenizerG.encode(seedword)

print(f'In BERT: "{seedword}" has index {seedidxB}')
print(f'In GPT2: "{seedword}" has index {seedidxG}')

In [ ]:
# Euclidean distances
eucDist_bert = np.sqrt(np.sum( (embeddingsB-embeddingsB[seedidxB,:].mean(axis=0))**2 ,axis=1))
eucDist_gpt2 = np.sqrt(np.sum( (embeddingsG-embeddingsG[seedidxG,:].mean(axis=0))**2 ,axis=1))

# sort and get top k
k = 15
topKidx_Bert = np.argsort(eucDist_bert)[:k] # [1:k+1] to exclude trivial self-distance
topKidx_gpt2 = np.argsort(eucDist_gpt2)[:k]

# and print
print('|        BERT        |        GPT2          |')
print('|--------------------|----------------------|')
for b,g in zip(topKidx_Bert,topKidx_gpt2):
  print(f'  {tokenizerB.decode([b]):>10} ({eucDist_bert[b]:.2f})  |  ({eucDist_gpt2[g]:4.2f}) {tokenizerG.decode([g])}')